In [ ]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

SAMPLE_RATE = 16000

In [ ]:
data_folders = ["90s_hiphop", "90s_rock", "2010s_pop", "classical", "country"]

# load trajectories
trajectories = dict()
for folder in data_folders:
    trajectories[folder] = np.load(f"data/{folder}_trajectories.npy", allow_pickle=True).item()

In [ ]:
# Crazy Torch Model
class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      # First fully connected layer
      self.fc1 = nn.Linear(13 * 768, 768)
      
      # Get outputs
      self.fc2 = nn.Linear(768,5)
      
    def forward(self, x):
      outputs = F.relu(self.fc1(x))
      outputs = F.sigmoid(self.fc2(outputs))
      return outputs
    
model = Net()
model.load_state_dict(torch.load('models/model_50000.pt'))

In [ ]:
model['fc1.weight'].shape

In [ ]:
from torchmetrics.functional import accuracy as torch_acc

asdf = ['2010s_pop']
for idx, folder in enumerate(asdf):
    tests = [trajectories[folder][i] for i in trajectories[folder].keys()]

    accs = []
    for i in tqdm(range(len(tests))):
        output = model(torch.tensor(tests[i]).flatten(start_dim=1).float())
        preds = torch.zeros_like(output)
        label = [0]*5
        label[2] = 1
        labels = torch.tensor([label]*preds.shape[0])

        preds.scatter_(1, torch.argmax(output, dim=1, keepdim=True), 1)
        acc = torch_acc(preds, labels, task='multiclass', num_classes=5)
        accs.append(acc)

        if acc < 0.8:
            print(i)

    plt.hist([a.item() for a in accs], bins=20)
    plt.title(folder)
    plt.show()

In [ ]:
song = trajectories['2010s_pop'][list(trajectories['2010s_pop'])[35]]

output = model(torch.tensor(song).flatten(start_dim=1).float())
preds = torch.zeros_like(output)
preds.scatter_(1, torch.argmax(output, dim=1, keepdim=True), 1)

label = [0]*5
label[2] = 1
labels = torch.tensor([label]*preds.shape[0])


In [ ]:
preds